In [2]:
import numpy as np
from scipy.optimize import root_scalar
from scipy.optimize import brentq
import matplotlib.pyplot as plt

# Functions

In [3]:
#parameters
t_1 = 0.425
t_2 = 0.05
t_3 = -0.025
t_4 = -0.075
kB = 8.617e-5
T=0.1

# k-grid
n_k = 150
kx = np.linspace(-np.pi, np.pi, n_k)
ky = np.linspace(-np.pi, np.pi, n_k)
KX, KY = np.meshgrid(kx, ky)



In [4]:
def epsilon(k_x,k_y,t_1,t_2,t_3,mu,epsi):
    return -2*t_2*(np.cos(k_x)+np.cos(k_y))-4*t_3*np.cos(k_x)*np.cos(k_y)-mu

def tx(k_x,k_y,t_1,epsi):
    return -4*t_1*np.exp(-1j*(k_x/2+k_y/2))*(np.cos(k_x/2)*np.cos(k_y/2)-epsi*np.sin(k_x/2)*np.sin(k_y/2))

def tz(k_x,k_y,t_4,epsi):
    return 4*t_4*np.sin(k_x)*np.sin(k_y)

In [5]:
def dispersion(k_x,k_y,sigma,N_A,N_B,U,mu,beta,epsi):
    return epsilon(k_x,k_y,t_1,t_2,t_3,mu,epsi)-U/4*sigma*(N_A+N_B)+beta*np.sqrt(np.abs(tx(k_x,k_y,t_1,epsi))**2+(tz(k_x,k_y,t_4,epsi)-U/4*sigma*(N_A-N_B))**2)

def fermi(E, T, mu=0):
    if T == 0:
        return np.where(E < mu, 1.0, 0.0)
    else:
        x = (E ) / (kB * T)
        x = np.clip(x, -700, 700)
        return 1/ (np.exp(x) + 1)

def energy_split(k_x,k_y,N_A,N_B,U,mu,epsi):
    E_up_plus=dispersion(k_x,k_y,1,N_A,N_B,U,mu,1,epsi)
    E_up_minus=dispersion(k_x,k_y,1,N_A,N_B,U,mu,-1,epsi)
    E_down_plus=dispersion(k_x,k_y,-1,N_A,N_B,U,mu,1,epsi)
    E_down_minus=dispersion(k_x,k_y,-1,N_A,N_B,U,mu,-1,epsi)
    return E_up_plus,E_up_minus,E_down_plus,E_down_minus

In [6]:
#eigenvectors

def a(sigma, N_A,N_B, U,epsi):
    return 1/np.sqrt(2)*np.sqrt(1+(tz(KX,KY,t_4,epsi)-U/4*sigma*(N_A-N_B))/np.sqrt(np.abs(tx(KX,KY,t_1,epsi))**2+(tz(KX,KY,t_4,epsi)-U/4*sigma*(N_A-N_B))**2))

def b(sigma, N_A,N_B, U,epsi):
    return -1/np.sqrt(2)*np.sqrt(1-(tz(KX,KY,t_4,epsi)-U/4*sigma*(N_A-N_B))/np.sqrt(np.abs(tx(KX,KY,t_1,epsi))**2+(tz(KX,KY,t_4,epsi)-U/4*sigma*(N_A-N_B))**2))

In [7]:
#lattice site A

def n_e(N_A,N_B,U,mu,T,epsi):
    E_up_plus,E_up_minus,E_down_plus,E_down_minus = energy_split(KX, KY, N_A,N_B, U, mu,epsi)
    f_up_plus = fermi(E_up_plus, T)
    f_up_minus = fermi(E_up_minus, T)
    f_down_plus = fermi(E_down_plus, T)
    f_down_minus = fermi(E_down_minus, T)
    return 1/n_k**2 * np.sum(a(1, N_A,N_B, U,epsi)**2*f_up_plus+a(-1, N_A,N_B, U,epsi)**2 *f_down_plus+b(1, N_A,N_B, U,epsi)**2*f_up_minus+b(-1, N_A,N_B, U,epsi)**2*f_down_minus)

def density_difference(mu, N_A,N_B, U, n_target, T,epsi):
    n = n_e(N_A,N_B, U, mu, T,epsi)
    return n - n_target

def order_param(N_A,N_B,U,mu,T,epsi):
    E_up_plus,E_up_minus,E_down_plus,E_down_minus = energy_split(KX, KY, N_A,N_B, U, mu,epsi)
    f_up_plus = fermi(E_up_plus, T)
    f_up_minus = fermi(E_up_minus, T)
    f_down_plus = fermi(E_down_plus, T)
    f_down_minus = fermi(E_down_minus, T)
    return 1/n_k**2 * np.sum(a(1, N_A,N_B, U,epsi)**2*f_up_plus-a(-1, N_A,N_B, U,epsi)**2 *f_down_plus+b(1, N_A,N_B, U,epsi)**2*f_up_minus-b(-1, N_A,N_B, U,epsi)**2*f_down_minus)

def n_up(N_A,N_B,U,mu,T,epsi):
    E_up_plus,E_up_minus,E_down_plus,E_down_minus = energy_split(KX, KY, N_A,N_B, U, mu,epsi)
    f_up_plus = fermi(E_up_plus, T)
    f_up_minus = fermi(E_up_minus, T)
    f_down_plus = fermi(E_down_plus, T)
    f_down_minus = fermi(E_down_minus, T)
    return 1/n_k**2 * np.sum(a(1, N_A,N_B, U,epsi)**2*f_up_plus+b(1, N_A,N_B, U,epsi)**2*f_up_minus)

def n_down(N_A,N_B,U,mu,T,epsi):
    E_up_plus,E_up_minus,E_down_plus,E_down_minus = energy_split(KX, KY, N_A,N_B, U, mu,epsi)
    f_up_plus = fermi(E_up_plus, T)
    f_up_minus = fermi(E_up_minus, T)
    f_down_plus = fermi(E_down_plus, T)
    f_down_minus = fermi(E_down_minus, T)
    return 1/n_k**2 * np.sum(a(-1, N_A,N_B, U,epsi)**2 *f_down_plus+b(-1, N_A,N_B, U,epsi)**2*f_down_minus)

In [8]:
#lattice site B

def n_e2(N_A,N_B,U,mu,T,epsi):
    E_up_plus,E_up_minus,E_down_plus,E_down_minus = energy_split(KX, KY, N_A,N_B, U, mu,epsi)
    f_up_plus = fermi(E_up_plus, T)
    f_up_minus = fermi(E_up_minus, T)
    f_down_plus = fermi(E_down_plus, T)
    f_down_minus = fermi(E_down_minus, T)
    return 1/n_k**2 * np.sum(b(1, N_A,N_B, U,epsi)**2*f_up_plus+b(-1, N_A,N_B, U,epsi)**2 *f_down_plus+a(1, N_A,N_B, U,epsi)**2*f_up_minus+a(-1, N_A,N_B, U,epsi)**2*f_down_minus)

def density_difference2(mu, N_A,N_B, U, n_target, T,epsi):
    n = n_e2(N_A,N_B, U, mu, T,epsi)
    return n - n_target

def order_param2(N_A,N_B,U,mu,T,epsi):
    E_up_plus,E_up_minus,E_down_plus,E_down_minus = energy_split(KX, KY, N_A,N_B, U, mu,epsi)
    f_up_plus = fermi(E_up_plus, T)
    f_up_minus = fermi(E_up_minus, T)
    f_down_plus = fermi(E_down_plus, T)
    f_down_minus = fermi(E_down_minus, T)
    return 1/n_k**2 * np.sum(b(1, N_A,N_B, U,epsi)**2*f_up_plus-b(-1, N_A,N_B, U,epsi)**2 *f_down_plus+a(1, N_A,N_B, U,epsi)**2*f_up_minus-a(-1, N_A,N_B, U,epsi)**2*f_down_minus)

def n_up2(N_A,N_B,U,mu,T,epsi):
    E_up_plus,E_up_minus,E_down_plus,E_down_minus = energy_split(KX, KY, N_A,N_B, U, mu,epsi)
    f_up_plus = fermi(E_up_plus, T)
    f_up_minus = fermi(E_up_minus, T)
    f_down_plus = fermi(E_down_plus, T)
    f_down_minus = fermi(E_down_minus, T)
    return 1/n_k**2 * np.sum(b(1, N_A,N_B, U,epsi)**2*f_up_plus+a(1, N_A,N_B, U,epsi)**2*f_up_minus)

def n_down2(N_A,N_B,U,mu,T,epsi):
    E_up_plus,E_up_minus,E_down_plus,E_down_minus = energy_split(KX, KY, N_A,N_B, U, mu,epsi)
    f_up_plus = fermi(E_up_plus, T)
    f_up_minus = fermi(E_up_minus, T)
    f_down_plus = fermi(E_down_plus, T)
    f_down_minus = fermi(E_down_minus, T)
    return 1/n_k**2 * np.sum(b(-1, N_A,N_B, U,epsi)**2 *f_down_plus+a(-1, N_A,N_B, U,epsi)**2*f_down_minus)

In [9]:
#total density difference

def density_difference_total(mu, N_A, N_B, U, n_target, T, epsi):
    n_A = n_e(N_A, N_B, U, mu, T, epsi)
    n_B = n_e2(N_A, N_B, U, mu, T, epsi)
    return ((n_A + n_B) / 2) - n_target


# Self consistent starting condition

In [13]:
#k grid
n_k = 150
kx = np.linspace(-np.pi, np.pi, n_k)
ky = np.linspace(-np.pi, np.pi, n_k)
KX, KY = np.meshgrid(kx, ky)

# Initial values
eps=0
x = 0

U = 3
N_A_current = 0.4
N_B_current = -0.4

#starting mu
n_0=1
N_0=0
U_0=1


try:
    mu_0 = brentq(density_difference_total, -10, 10, args=(N_0,-N_0 ,U_0, n_0, T,eps), xtol=1e-5)
except ValueError:
    print("mu not found")

print(mu_0)
mu_current = mu_0



for i in range(1000):
    # Compute updated order parameters
    N_A_new = x * N_A_current + (1 - x) * order_param(N_A_current, N_B_current, U, mu_current, T,eps)
    N_B_new = x * N_B_current + (1 - x) * order_param2(N_A_current, N_B_current, U, mu_current, T,eps)

    # Update mu to maintain total density ~1
    try:
        mu_new = brentq(density_difference_total, -10, 10, args=(N_A_new, N_B_new, U, n_0, T,eps), xtol=1e-5)
    except ValueError:
        print("mu not found")
        break

    # Check total density
    n_new_A = n_e(N_A_new, N_B_new, U, mu_new, T, eps)
    n_new_B = n_e2(N_A_new, N_B_new, U, mu_new, T, eps)
    n_total = (n_new_A + n_new_B) / 2

    #print(f"step {i+1}: N_A = {N_A_new:.3f}, N_B = {N_B_new:.3f}, μ = {mu_new:.3f}, n = {n_total:.3f}")

    # Check convergence
    if (np.abs(N_A_new - N_A_current) +
        np.abs(N_B_new - N_B_current) +
        np.abs(n_total - n_0)) < 1e-5:
        print(f"✅ Converged in {i+1} steps.")
        break

    # Update for next step
    N_A_current = N_A_new
    N_B_current = N_B_new
    mu_current = mu_new


print("\n📌 Final self-consistent results:")
print(f"N_A = {N_A_current:.6f}")
print(f"N_B = {N_B_current:.6f}")
print(f"μ   = {mu_current:.6f}")
print(f"n   = {(n_e(N_A_current, N_B_current, U, mu_current, T,eps) + n_e2(N_A_current, N_B_current, U, mu_current, T,eps)) / 2:.6f}")
print(f"N_A + N_B = {N_A_current + N_B_current:.6e}")

0.12798745079306958
✅ Converged in 9 steps.

📌 Final self-consistent results:
N_A = 0.866006
N_B = -0.866006
μ   = 1.198803
n   = 1.000000
N_A + N_B = 0.000000e+00


# Runge Kutta Method

In [ ]:
#derivative of order parameters with respect to chemical potential
#the dispersion depends on the chemical potential, so we need the derivative of the fermi function

def fermi_derivative ():
    return 1



N_A0 = 0.866006
N_B0 = -0.866006
μ0   = 1.198803
U=3
epsilon=0.05

